In [2]:
import torch 
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib  inline

In [3]:
words = open('names.txt', 'r').read().splitlines()
len(words)

32033

In [4]:
chars = sorted(list(set("".join(words))))
stoi = {s: i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}
vocab_size = len(itos)
print(itos, vocab_size)


{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'} 27


In [5]:
# build the dataset
block_size = 3 # how many characters do we take to predict next one?

def build_dataset(words):
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])


torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [ ]:
# MLP revised 
n_embd = 10 # dimentinality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP
g = torch.Generator().manual_seed(5462384)
C = torch.randn((vocab_size, n_embd),             generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3) / ((n_embd * block_size) ** 0.5)
# b1 = torch.randn(n_hidden,                        generator=g) * 0.01
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.01
b2 = torch.randn(vocab_size,                      generator=g) * 0 

bngains = torch.ones((1, n_hidden))
bnbias = torch.zeros((1, n_hidden))

bnmean_running = torch.zeros((1, n_hidden))
bnstd_running  = torch.ones((1, n_hidden))



parameters = [C, W1, W2, b2, bngains, bnbias]

print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

12297


In [ ]:
# optimization
max_steps = 100000
batch_size = 32
lossi = []

for i in range(max_steps):
    #minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]

    # forward pass
    emb = C[Xb] # embed characters into vectors 
    embcat  = emb.view(emb.shape[0], -1) # concatenete the vectors

    # BatchNorm Layer 
    hpreact = embcat @ W1 #+ b1 # hidden layer pre-activation
    bnmeani = hpreact.mean(dim=0, keepdim=True)
    bnstdi = hpreact.std(dim=0, keepdim=True)
    hpreact = bngains * (hpreact - bnmeani) / bnstdi + bnbias

    with torch.no_grad():
        bnmean_running = 0.999 * bnmean_running + 0.001 * bnmeani
        bnstd_running = 0.999 * bnstd_running + 0.001 * bnstdi

    # Non-linearity 
    h = torch.tanh(hpreact) # hidden layer
    logits = h @ W2 + b2 # output layer
    loss = F.cross_entropy(logits, Yb) # loss function 

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # upadate
    lr = 0.1 if i < 1000 else 0.01 # step learning rate decay
    for p in parameters:
        p.data += -lr * p.grad
    
    # track stats
    if i % 1000 == 0:
        print(f'{i:7d} / {max_steps:7d}: {loss.item():4f}')

    lossi.append(loss.log10().item())

      0 /  100000: 2.385844
   1000 /  100000: 2.659806
   2000 /  100000: 2.252975
   3000 /  100000: 2.516990
   4000 /  100000: 2.294872
   5000 /  100000: 2.214381
   6000 /  100000: 2.172044
   7000 /  100000: 2.170521
   8000 /  100000: 2.031156
   9000 /  100000: 2.475476
  10000 /  100000: 2.337784
  11000 /  100000: 2.162754
  12000 /  100000: 2.205140
  13000 /  100000: 2.348062
  14000 /  100000: 2.479218
  15000 /  100000: 2.390606
  16000 /  100000: 1.963026
  17000 /  100000: 2.240380
  18000 /  100000: 2.429080
  19000 /  100000: 2.496686
  20000 /  100000: 2.188104
  21000 /  100000: 2.247715
  22000 /  100000: 2.462376
  23000 /  100000: 2.054964
  24000 /  100000: 2.307641
  25000 /  100000: 2.293348
  26000 /  100000: 1.907834
  27000 /  100000: 2.587777
  28000 /  100000: 2.103356
  29000 /  100000: 2.321729
  30000 /  100000: 2.313060
  31000 /  100000: 2.186671
  32000 /  100000: 2.445682
  33000 /  100000: 2.199970
  34000 /  100000: 1.866409
  35000 /  100000: 2

In [98]:
hpreact.mean(dim=0, keepdim=True).shape

torch.Size([1, 200])

In [89]:
hpreact.std(dim=0, keepdim=True).shape

torch.Size([1, 200])

In [107]:
with torch.no_grad():
    emb = C[Xtr]
    embcat = emb.view((emb.shape[0], -1))
    hpreact = embcat @ W1 + b1
    bnmean = hpreact.mean(dim=0, keepdim=True)
    bnstd  = hpreact.std(dim=0, keepdim=True)

In [116]:
@torch.no_grad()
def split_loss(split):
    x, y =  {
        'train': (Xtr, Ytr), 
        'val':   (Xdev, Ydev), 
        'test':   (Xte, Xte),
    }[split]
    emb = C[x]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 + b1
    hpreact = bngains * (hpreact -  bnmean_running) / bnstd_running + bnbias
    h = torch.tanh(hpreact)
    losits = h @ W2 + b2
    loss = F.cross_entropy(losits, y)
    print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.161543369293213
val 2.1825852394104004


torch.Size([2, 3])


tensor([[5, 7, 9]])